In [4]:
import tensorflow as tf
from prosit_t.wandb_agent.train_utils import get_tfdatasets_padded_filtered
import os
import argparse
import yaml
from dlomix.losses import masked_spectral_distance
import wandb
from prosit_t.models import PrositTransformerV2
from prosit_t.eval import prosit_transformer_eval
from wandb.keras import WandbCallback
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from prosit_t.constants import ALPHABET_UNMOD

In [5]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# physical_devices = tf.config.list_physical_devices("GPU")
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [6]:
project_name = "transforming-prosit-full-dataset"
run = wandb.init(project=project_name)

In [7]:
artifact_path = "prosit-compms/transforming-prosit-full-dataset/model-astral-monkey-20:v18"
model = run.use_artifact(artifact_path)
model_dir = model.download()

wandb:   4 of 4 files downloaded.  


In [8]:
model_loaded = tf.keras.models.load_model(model_dir, custom_objects={
    "masked_spectral_distance": masked_spectral_distance,
})

2023-11-16 08:40:09.722189: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-16 08:40:12.659167: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43648 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:81:00.0, compute capability: 8.6


In [10]:
data_config = {
    "data_source": {
        "train": "/cmnfs/proj/prosit/Transformer/all_unmod_train.parquet",
          "val": "/cmnfs/proj/prosit/Transformer/all_unmod_test.parquet"
    },
    "seq_length": 30,
    "batch_size": 1024
          
}

In [11]:
train_dataset, val_dataset = get_tfdatasets_padded_filtered(data_config)

In [12]:
cb_wandb = WandbCallback()

callback_earlystopping = EarlyStopping(
    monitor="val_loss",
    patience=8,
    min_delta=0.0001,
    restore_best_weights=True,
    verbose=1,
)

callback_reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.8,
    patience=4,
)
callbacks = [cb_wandb, callback_earlystopping, callback_reduce_lr]

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [ ]:
model_loaded.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100,
    callbacks=callbacks,
)

Epoch 1/100


2023-11-16 08:50:45.224099: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA A40" frequency: 1740 num_cores: 84 environment { key: "architecture" value: "8.6" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 6291456 shared_memory_size_per_multiprocessor: 102400 memory_size: 45768966144 bandwidth: 696096000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
2023-11-16 08:50:46.669228: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


12178/12178 [==============================] - ETA: 0s - loss: 0.0987

2023-11-16 09:28:48.107562: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA A40" frequency: 1740 num_cores: 84 environment { key: "architecture" value: "8.6" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 6291456 shared_memory_size_per_multiprocessor: 102400 memory_size: 45768966144 bandwidth: 696096000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.


INFO:tensorflow:Assets written to: /cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20231116_083957-jagiqmx1/files/model-best/assets


INFO:tensorflow:Assets written to: /cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20231116_083957-jagiqmx1/files/model-best/assets
wandb: Adding directory to artifact (/cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20231116_083957-jagiqmx1/files/model-best)... Done. 0.3s


12178/12178 [==============================] - 2514s 206ms/step - loss: 0.0987 - val_loss: 0.1154 - lr: 3.2768e-05
Epoch 2/100
 4657/12178 [==========>...................] - ETA: 23:28 - loss: 0.1072

In [14]:
transformer.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 30)]         0           []                               
                                                                                                  
 string_lookup (StringLookup)   (None, 30)           0           ['input_4[0][0]']                
                                                                                                  
 positional_embedding (Position  (None, 30, 64)      1472        ['string_lookup[1][0]']          
 alEmbedding)                                                                                     
                                                                                                  
 transformer_encoder (Transform  (None, 30, 64)      2157824     ['positional_embedding[1][0